In [ ]:

from google.colab import files

uploaded = files.upload()  

pdf_path = list(uploaded.keys())[0]
print("Uploaded PDF:", pdf_path)


In [ ]:

!pip install pymupdf


In [ ]:

import fitz  
import os

output_folder = "extracted_images"
os.makedirs(output_folder, exist_ok=True)

doc = fitz.open(pdf_path)

image_count = 0

for page_index in range(len(doc)):
    page = doc[page_index]
    image_list = page.get_images(full=True)

    print(f"Page {page_index + 1}: {len(image_list)} image(s)")

    for img_index, img in enumerate(image_list, start=1):
        xref = img[0]
        pix = fitz.Pixmap(doc, xref)

        if pix.colorspace.name != "DeviceRGB" and pix.colorspace.name != "DeviceGray":
            pix = fitz.Pixmap(fitz.csRGB, pix)

        image_count += 1
        image_path = os.path.join(
            output_folder,
            f"page_{page_index + 1}_img_{img_index}.png"
        )
        pix.save(image_path)
        pix = None  

print(f"\nTotal images extracted: {image_count}")
print("Images saved in folder:", output_folder)


In [ ]:
import shutil

shutil.make_archive("extracted_images", "zip", "extracted_images")
print("Folder zipped successfully!")


In [ ]:
from google.colab import files
files.download("extracted_images.zip")


In [ ]:
from IPython.display import display
from PIL import Image
import glob

image_files = sorted(glob.glob(output_folder + "/*.png"))[:5]

for img_path in image_files:
    print(img_path)
    display(Image.open(img_path))


In [ ]:
pip install easyocr opencv-python pillow numpy


In [ ]:
import easyocr
import cv2
for img_path in image_files:
  reader = easyocr.Reader(['de'], gpu=False)
  results = reader.readtext(img_path)


In [ ]:
for box, text, conf in results:
    print(f"{text}  (confidence: {conf:.2f})")


In [ ]:
import numpy as np

img = cv2.imread(image_path)
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

_, thresh = cv2.threshold(gray, 200, 255, cv2.THRESH_BINARY_INV)

contours, _ = cv2.findContours(
    thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE
)

boxes = []
for cnt in contours:
    x, y, w, h = cv2.boundingRect(cnt)
    if w > 150 and h > 50:
        boxes.append((x, y, w, h))


In [ ]:
structured_boxes = []

for (bbox, text, conf) in results:
    structured_boxes.append({
        "box_position": bbox,  
        "text": text,
        "confidence": conf,
        "type": None
    })


In [ ]:
def classify_node(text):
    if "Falls JA" in text:
        return "YES"
    if "Falls NEIN" in text:
        return "NO"
    return "NODE"


for box in structured_boxes:
    box["type"] = classify_node(box["text"])



In [ ]:

for i, box in enumerate(structured_boxes):

    print(f"\nBOX {i+1}")
    print("Position:", box["box_position"])
    print("Text:", box["text"])
    print("Type:", box["type"])



In [ ]:
import json
import numpy as np 
json_ready = []
for box in structured_boxes:
    x_coords = [int(p[0]) for p in box["box_position"]]
    y_coords = [int(p[1]) for p in box["box_position"]]

    x_min = min(x_coords)
    y_min = min(y_coords)
    x_max = max(x_coords)
    y_max = max(y_coords)

    width = x_max - x_min
    height = y_max - y_min

    json_ready.append({
        "box_position": {
            "x": x_min,
            "y": y_min,
            "width": width,
            "height": height
        },
        "text": box["text"],
        "type": box["type"]
    })

json_data = json.dumps(json_ready, indent=4, ensure_ascii=False)
print(json_data)

with open("structured_boxes.json", "w", encoding="utf-8") as f:
    f.write(json_data)


In [ ]:
import json

data = [
    {
        "box_position": {
            "x": 347,
            "y": 13,
            "width": 110,
            "height": 16
        },
        "text": "Patient beatmet?",
        "type": "NODE"
    },
    {
        "box_position": {
            "x": 353,
            "y": 43,
            "width": 38,
            "height": 14
        },
        "text": "Falls JA",
        "type": "YES"
    },
    {
        "box_position": {
            "x": 293,
            "y": 73,
            "width": 216,
            "height": 14
        },
        "text": "Ist das Bewusstsein des Patienten",
        "type": "NODE"
    },
    {
        "box_position": {
            "x": 609,
            "y": 73,
            "width": 38,
            "height": 12
        },
        "text": "Falls JA",
        "type": "YES"
    },
    {
        "box_position": {
            "x": 537,
            "y": 85,
            "width": 24,
            "height": 12
        },
        "text": "Falls",
        "type": "NODE"
    },
    {
        "box_position": {
            "x": 368,
            "y": 87,
            "width": 63,
            "height": 21
        },
        "text": "adäquat?",
        "type": "NODE"
    },
    {
        "box_position": {
            "x": 536,
            "y": 100,
            "width": 26,
            "height": 8
        },
        "text": "NEr",
        "type": "NODE"
    },
    {
        "box_position": {
            "x": 339,
            "y": 107,
            "width": 122,
            "height": 16
        },
        "text": "(RASS-Score >=-3)",
        "type": "NODE"
    },
    {
        "box_position": {
            "x": 605,
            "y": 107,
            "width": 142,
            "height": 18
        },
        "text": "Zunächst primäre und",
        "type": "NODE"
    },
    {
        "box_position": {
            "x": 597,
            "y": 125,
            "width": 156,
            "height": 18
        },
        "text": "supportive Therapie, bis",
        "type": "NODE"
    },
    {
        "box_position": {
            "x": 353,
            "y": 143,
            "width": 36,
            "height": 14
        },
        "text": "Falls JA",
        "type": "YES"
    },
    {
        "box_position": {
            "x": 601,
            "y": 142,
            "width": 150,
            "height": 19
        },
        "text": "sich die Wahrnehmung",
        "type": "NODE"
    },
    {
        "box_position": {
            "x": 536,
            "y": 158,
            "width": 22,
            "height": 8
        },
        "text": "Fallls",
        "type": "NODE"
    },
    {
        "box_position": {
            "x": 607,
            "y": 161,
            "width": 136,
            "height": 14
        },
        "text": "und das Bewusstsein",
        "type": "NODE"
    },
    {
        "box_position": {
            "x": 536,
            "y": 172,
            "width": 24,
            "height": 8
        },
        "text": "NEIN",
        "type": "NODE"
    },
    {
        "box_position": {
            "x": 315,
            "y": 173,
            "width": 170,
            "height": 18
        },
        "text": "Ist das kognitive Level des",
        "type": "NODE"
    },
    {
        "box_position": {
            "x": 639,
            "y": 177,
            "width": 74,
            "height": 16
        },
        "text": "verbessern",
        "type": "NODE"
    },
    {
        "box_position": {
            "x": 339,
            "y": 191,
            "width": 88,
            "height": 14
        },
        "text": "Patienten adä",
        "type": "NODE"
    },
    {
        "box_position": {
            "x": 365,
            "y": 207,
            "width": 70,
            "height": 16
        },
        "text": "(CAM-ICU)",
        "type": "NODE"
    },
    {
        "box_position": {
            "x": 353,
            "y": 243,
            "width": 38,
            "height": 14
        },
        "text": "Falls JA",
        "type": "YES"
    },
    {
        "box_position": {
            "x": 367,
            "y": 273,
            "width": 64,
            "height": 14
        },
        "text": "Evaluiere:",
        "type": "NODE"
    },
    {
        "box_position": {
            "x": 336,
            "y": 290,
            "width": 79,
            "height": 19
        },
        "text": "Hörfähigkeit",
        "type": "NODE"
    },
    {
        "box_position": {
            "x": 337,
            "y": 311,
            "width": 70,
            "height": 16
        },
        "text": "Sehschärfe",
        "type": "NODE"
    },
    {
        "box_position": {
            "x": 337,
            "y": 333,
            "width": 150,
            "height": 16
        },
        "text": "Sprache (Muttersprache;",
        "type": "NODE"
    },
    {
        "box_position": {
            "x": 337,
            "y": 353,
            "width": 66,
            "height": 16
        },
        "text": "Präferenz)",
        "type": "NODE"
    },
    {
        "box_position": {
            "x": 348,
            "y": 410,
            "width": 103,
            "height": 21
        },
        "text": "Beurteilung der",
        "type": "NODE"
    },
    {
        "box_position": {
            "x": 331,
            "y": 431,
            "width": 138,
            "height": 16
        },
        "text": "Funktionstähigkeiten",
        "type": "NODE"
    },
    {
        "box_position": {
            "x": 377,
            "y": 463,
            "width": 52,
            "height": 12
        },
        "text": "Falls NEIN",
        "type": "NO"
    },
    {
        "box_position": {
            "x": 73,
            "y": 483,
            "width": 154,
            "height": 16
        },
        "text": "Sind die feinmotorische",
        "type": "NODE"
    },
    {
        "box_position": {
            "x": 565,
            "y": 485,
            "width": 165,
            "height": 14
        },
        "text": "Sind dic oral-motorischen",
        "type": "NODE"
    },
    {
        "box_position": {
            "x": 89,
            "y": 501,
            "width": 124,
            "height": 16
        },
        "text": "Fähigkeiten intakt?",
        "type": "NODE"
    },
    {
        "box_position": {
            "x": 587,
            "y": 501,
            "width": 124,
            "height": 18
        },
        "text": "Fähigkeiten intakt?",
        "type": "NODE"
    },
    {
        "box_position": {
            "x": 373,
            "y": 513,
            "width": 54,
            "height": 12
        },
        "text": "Falls NEIN",
        "type": "NO"
    },
    {
        "box_position": {
            "x": 273,
            "y": 523,
            "width": 26,
            "height": 14
        },
        "text": "Falls",
        "type": "NODE"
    },
    {
        "box_position": {
            "x": 530,
            "y": 528,
            "width": 32,
            "height": 8
        },
        "text": "Fallsla",
        "type": "NODE"
    },
    {
        "box_position": {
            "x": 65,
            "y": 531,
            "width": 38,
            "height": 14
        },
        "text": "Falls JA",
        "type": "YES"
    },
    {
        "box_position": {
            "x": 273,
            "y": 537,
            "width": 28,
            "height": 12
        },
        "text": "NEN",
        "type": "NODE"
    },
    {
        "box_position": {
            "x": 459,
            "y": 547,
            "width": 120,
            "height": 16
        },
        "text": "Hat der Patient ein",
        "type": "NODE"
    },
    {
        "box_position": {
            "x": 641,
            "y": 547,
            "width": 128,
            "height": 16
        },
        "text": "Hat der Patient eine",
        "type": "NODE"
    },
    {
        "box_position": {
            "x": 45,
            "y": 561,
            "width": 110,
            "height": 16
        },
        "text": "Kann der Patient",
        "type": "NODE"
    },
    {
        "box_position": {
            "x": 275,
            "y": 565,
            "width": 54,
            "height": 14
        },
        "text": "Sind dic",
        "type": "NODE"
    },
    {
        "box_position": {
            "x": 469,
            "y": 564,
            "width": 100,
            "height": 14
        },
        "text": "Tracheostoma?",
        "type": "NODE"
    },
    {
        "box_position": {
            "x": 663,
            "y": 565,
            "width": 84,
            "height": 16
        },
        "text": "orale Sonde?",
        "type": "NODE"
    },
    {
        "box_position": {
            "x": 63,
            "y": 579,
            "width": 72,
            "height": 14
        },
        "text": "schreiben?",
        "type": "NODE"
    },
    {
        "box_position": {
            "x": 244,
            "y": 581,
            "width": 115,
            "height": 18
        },
        "text": "grobmotorischen",
        "type": "NODE"
    },
    {
        "box_position": {
            "x": 461,
            "y": 591,
            "width": 36,
            "height": 14
        },
        "text": "FalsIA",
        "type": "NODE"
    },
    {
        "box_position": {
            "x": 714,
            "y": 598,
            "width": 32,
            "height": 8
        },
        "text": "Fallsla",
        "type": "NODE"
    },
    {
        "box_position": {
            "x": 241,
            "y": 599,
            "width": 124,
            "height": 16
        },
        "text": "Fähigkeiten intakt?",
        "type": "NODE"
    },
    {
        "box_position": {
            "x": 21,
            "y": 613,
            "width": 36,
            "height": 14
        },
        "text": "FalsJA",
        "type": "NODE"
    },
    {
        "box_position": {
            "x": 125,
            "y": 611,
            "width": 24,
            "height": 14
        },
        "text": "Falls",
        "type": "NODE"
    },
    {
        "box_position": {
            "x": 447,
            "y": 613,
            "width": 124,
            "height": 16
        },
        "text": "Sind die laryngealen",
        "type": "NODE"
    },
    {
        "box_position": {
            "x": 123,
            "y": 625,
            "width": 30,
            "height": 12
        },
        "text": "NEIN",
        "type": "NODE"
    },
    {
        "box_position": {
            "x": 629,
            "y": 623,
            "width": 54,
            "height": 14
        },
        "text": "Falls NEIN",
        "type": "NO"
    },
    {
        "box_position": {
            "x": 446,
            "y": 629,
            "width": 93,
            "height": 18
        },
        "text": "Strukturen des",
        "type": "NODE"
    },
    {
        "box_position": {
            "x": 57,
            "y": 653,
            "width": 38,
            "height": 14
        },
        "text": "Stelle",
        "type": "NODE"
    },
    {
        "box_position": {
            "x": 205,
            "y": 645,
            "width": 140,
            "height": 16
        },
        "text": "Kommunikationsboard",
        "type": "NODE"
    },
    {
        "box_position": {
            "x": 447,
            "y": 646,
            "width": 106,
            "height": 18
        },
        "text": "Patienten intakt?",
        "type": "NODE"
    },
    {
        "box_position": {
            "x": 41,
            "y": 669,
            "width": 70,
            "height": 16
        },
        "text": "Papier und",
        "type": "NODE"
    },
    {
        "box_position": {
            "x": 203,
            "y": 663,
            "width": 38,
            "height": 18
        },
        "text": "VOCA",
        "type": "NODE"
    },
    {
        "box_position": {
            "x": 461,
            "y": 671,
            "width": 36,
            "height": 14
        },
        "text": "@1",
        "type": "NODE"
    },
    {
        "box_position": {
            "x": 61,
            "y": 687,
            "width": 30,
            "height": 14
        },
        "text": "Stitt",
        "type": "NODE"
    },
    {
        "box_position": {
            "x": 205,
            "y": 685,
            "width": 148,
            "height": 20
        },
        "text": "Computergestützte AAC",
        "type": "NODE"
    },
    {
        "box_position": {
            "x": 449,
            "y": 689,
            "width": 132,
            "height": 16
        },
        "text": "Kann der Patient eine",
        "type": "NODE"
    },
    {
        "box_position": {
            "x": 29,
            "y": 705,
            "width": 26,
            "height": 14
        },
        "text": "Zur",
        "type": "NODE"
    },
    {
        "box_position": {
            "x": 54,
            "y": 702,
            "width": 69,
            "height": 20
        },
        "text": "Verfugung",
        "type": "NODE"
    },
    {
        "box_position": {
            "x": 449,
            "y": 707,
            "width": 110,
            "height": 14
        },
        "text": "Deflation des Cuff",
        "type": "NODE"
    },
    {
        "box_position": {
            "x": 449,
            "y": 725,
            "width": 68,
            "height": 14
        },
        "text": "tolerieren?",
        "type": "NODE"
    },
    {
        "box_position": {
            "x": 391,
            "y": 743,
            "width": 24,
            "height": 12
        },
        "text": "Falls",
        "type": "NODE"
    },
    {
        "box_position": {
            "x": 604,
            "y": 748,
            "width": 48,
            "height": 8
        },
        "text": "FallsNFIN",
        "type": "NODE"
    },
    {
        "box_position": {
            "x": 217,
            "y": 751,
            "width": 38,
            "height": 14
        },
        "text": "Falls JA",
        "type": "YES"
    },
    {
        "box_position": {
            "x": 389,
            "y": 755,
            "width": 30,
            "height": 12
        },
        "text": "NEN",
        "type": "NODE"
    },
    {
        "box_position": {
            "x": 487,
            "y": 749,
            "width": 38,
            "height": 12
        },
        "text": "EIst4",
        "type": "NODE"
    },
    {
        "box_position": {
            "x": 478,
            "y": 768,
            "width": 51,
            "height": 17
        },
        "text": "Einweg-",
        "type": "NODE"
    },
    {
        "box_position": {
            "x": 589,
            "y": 767,
            "width": 96,
            "height": 18
        },
        "text": "Hat der Patient",
        "type": "NODE"
    },
    {
        "box_position": {
            "x": 109,
            "y": 791,
            "width": 124,
            "height": 18
        },
        "text": "Computergestützte",
        "type": "NODE"
    },
    {
        "box_position": {
            "x": 287,
            "y": 793,
            "width": 74,
            "height": 16
        },
        "text": "Blick- oder",
        "type": "NODE"
    },
    {
        "box_position": {
            "x": 479,
            "y": 787,
            "width": 78,
            "height": 16
        },
        "text": "Sprechventil",
        "type": "NODE"
    },
    {
        "box_position": {
            "x": 589,
            "y": 787,
            "width": 106,
            "height": 18
        },
        "text": "hartnäckige oder",
        "type": "NODE"
    },
    {
        "box_position": {
            "x": 109,
            "y": 809,
            "width": 30,
            "height": 14
        },
        "text": "AAC",
        "type": "NODE"
    },
    {
        "box_position": {
            "x": 589,
            "y": 805,
            "width": 116,
            "height": 14
        },
        "text": "reichliche Sekrete?",
        "type": "NODE"
    },
    {
        "box_position": {
            "x": 287,
            "y": 811,
            "width": 154,
            "height": 18
        },
        "text": "Augenzwinkergesteuert",
        "type": "NODE"
    },
    {
        "box_position": {
            "x": 169,
            "y": 827,
            "width": 26,
            "height": 14
        },
        "text": "mit",
        "type": "NODE"
    },
    {
        "box_position": {
            "x": 297,
            "y": 824,
            "width": 125,
            "height": 24
        },
        "text": "computergestutzte",
        "type": "NODE"
    },
    {
        "box_position": {
            "x": 707,
            "y": 833,
            "width": 38,
            "height": 14
        },
        "text": "Falls JA",
        "type": "YES"
    },
    {
        "box_position": {
            "x": 109,
            "y": 843,
            "width": 148,
            "height": 16
        },
        "text": "Einzelschaltersteuerun",
        "type": "NODE"
    },
    {
        "box_position": {
            "x": 287,
            "y": 843,
            "width": 30,
            "height": 16
        },
        "text": "AAC",
        "type": "NODE"
    },
    {
        "box_position": {
            "x": 489,
            "y": 841,
            "width": 118,
            "height": 16
        },
        "text": "Sprecheinsatz der",
        "type": "NODE"
    },
    {
        "box_position": {
            "x": 631,
            "y": 843,
            "width": 24,
            "height": 12
        },
        "text": "Falls",
        "type": "NODE"
    },
    {
        "box_position": {
            "x": 629,
            "y": 855,
            "width": 30,
            "height": 12
        },
        "text": "NEIN",
        "type": "NODE"
    },
    {
        "box_position": {
            "x": 123,
            "y": 861,
            "width": 32,
            "height": 14
        },
        "text": "oder",
        "type": "NODE"
    },
    {
        "box_position": {
            "x": 154,
            "y": 859,
            "width": 99,
            "height": 20
        },
        "text": "Blicksteuerung",
        "type": "NODE"
    },
    {
        "box_position": {
            "x": 497,
            "y": 859,
            "width": 102,
            "height": 14
        },
        "text": "Trachealkanüle",
        "type": "NODE"
    },
    {
        "box_position": {
            "x": 680,
            "y": 857,
            "width": 91,
            "height": 20
        },
        "text": "Elektrolarynx",
        "type": "NODE"
    },
    {
        "box_position": {
            "x": 53,
            "y": 901,
            "width": 698,
            "height": 20
        },
        "text": "Dokumentiere den Kommunikationsbedarf und den Erfolg von Kommunikationsinterventionen in der",
        "type": "NODE"
    },
    {
        "box_position": {
            "x": 351,
            "y": 921,
            "width": 98,
            "height": 16
        },
        "text": "Patientenakte",
        "type": "NODE"
    },
    {
        "box_position": {
            "x": 86,
            "y": 936,
            "width": 630,
            "height": 24
        },
        "text": "(bei mangelnder Effektivität und ausbleibendem Erfolg sollte eine alternative augmentative",
        "type": "NODE"
    },
    {
        "box_position": {
            "x": 241,
            "y": 959,
            "width": 318,
            "height": 18
        },
        "text": "Kommunikationsmethode angeboten werden)",
        "type": "NODE"
    },
    {
        "box_position": {
            "x": 422,
            "y": 189,
            "width": 40,
            "height": 18
        },
        "text": "quat?",
        "type": "NODE"
    }
]

def linearize_flowchart(json_data):
    sorted_nodes = sorted(json_data, key=lambda x: x['box_position']['y'])

    context = "Algorithmus für die Kommunikation mit beatmeten Patienten:\n"
    for node in sorted_nodes:
        context += f"- {node['text']}\n"
    return context

flowchart_text = linearize_flowchart(data)

In [ ]:
!pip install -q -U google-generativeai

In [ ]:
import google.generativeai as genai

API_KEY = ""     #Enter API Key
genai.configure(api_key=API_KEY)


print("Checking available models...")
available_models = [m.name for m in genai.list_models() if 'generateContent' in m.supported_generation_methods]

if not available_models:
    print("No models found. Check your API key or billing status.")
else:
    selected_model = next((m for m in available_models if "flash" in m), available_models[0])
    print(f"Using model: {selected_model}")

    model = genai.GenerativeModel(selected_model)

    json_data = json_data = [
    {"box_position": {"x": 347, "y": 13, "width": 110, "height": 16}, "text": "Patient beatmet?", "type": "NODE"},
    {"box_position": {"x": 353, "y": 43, "width": 38, "height": 14}, "text": "Falls JA", "type": "YES"},
    {"box_position": {"x": 293, "y": 73, "width": 216, "height": 14}, "text": "Ist das Bewusstsein des Patienten", "type": "NODE"},
    {"box_position": {"x": 609, "y": 73, "width": 38, "height": 12}, "text": "Falls JA", "type": "YES"},
    {"box_position": {"x": 537, "y": 85, "width": 24, "height": 12}, "text": "Falls", "type": "NODE"},
    {"box_position": {"x": 368, "y": 87, "width": 63, "height": 21}, "text": "adäquat?", "type": "NODE"},
    {"box_position": {"x": 536, "y": 100, "width": 26, "height": 8}, "text": "NEr", "type": "NODE"},
    {"box_position": {"x": 339, "y": 107, "width": 122, "height": 16}, "text": "(RASS-Score >=-3)", "type": "NODE"},
    {"box_position": {"x": 605, "y": 107, "width": 142, "height": 18}, "text": "Zunächst primäre und", "type": "NODE"},
    {"box_position": {"x": 597, "y": 125, "width": 156, "height": 18}, "text": "supportive Therapie, bis", "type": "NODE"},
    {"box_position": {"x": 353, "y": 143, "width": 36, "height": 14}, "text": "Falls JA", "type": "YES"},
    {"box_position": {"x": 601, "y": 142, "width": 150, "height": 19}, "text": "sich die Wahrnehmung", "type": "NODE"},
    {"box_position": {"x": 536, "y": 158, "width": 22, "height": 8}, "text": "Fallls", "type": "NODE"},
    {"box_position": {"x": 607, "y": 161, "width": 136, "height": 14}, "text": "und das Bewusstsein", "type": "NODE"},
    {"box_position": {"x": 536, "y": 172, "width": 24, "height": 8}, "text": "NEIN", "type": "NODE"},
    {"box_position": {"x": 315, "y": 173, "width": 170, "height": 18}, "text": "Ist das kognitive Level des", "type": "NODE"},
    {"box_position": {"x": 639, "y": 177, "width": 74, "height": 16}, "text": "verbessern", "type": "NODE"},
    {"box_position": {"x": 339, "y": 191, "width": 88, "height": 14}, "text": "Patienten adä", "type": "NODE"},
    {"box_position": {"x": 365, "y": 207, "width": 70, "height": 16}, "text": "(CAM-ICU)", "type": "NODE"},
    {"box_position": {"x": 353, "y": 243, "width": 38, "height": 14}, "text": "Falls JA", "type": "YES"},
    {"box_position": {"x": 367, "y": 273, "width": 64, "height": 14}, "text": "Evaluiere:", "type": "NODE"},
    {"box_position": {"x": 336, "y": 290, "width": 79, "height": 19}, "text": "Hörfähigkeit", "type": "NODE"},
    {"box_position": {"x": 337, "y": 311, "width": 70, "height": 16}, "text": "Sehschärfe", "type": "NODE"},
    {"box_position": {"x": 337, "y": 333, "width": 150, "height": 16}, "text": "Sprache (Muttersprache;", "type": "NODE"},
    {"box_position": {"x": 337, "y": 353, "width": 66, "height": 16}, "text": "Präferenz)", "type": "NODE"},
    {"box_position": {"x": 348, "y": 410, "width": 103, "height": 21}, "text": "Beurteilung der", "type": "NODE"},
    {"box_position": {"x": 331, "y": 431, "width": 138, "height": 16}, "text": "Funktionstähigkeiten", "type": "NODE"},
    {"box_position": {"x": 377, "y": 463, "width": 52, "height": 12}, "text": "Falls NEIN", "type": "NO"},
    {"box_position": {"x": 73, "y": 483, "width": 154, "height": 16}, "text": "Sind die feinmotorische", "type": "NODE"},
    {"box_position": {"x": 565, "y": 485, "width": 165, "height": 14}, "text": "Sind dic oral-motorischen", "type": "NODE"},
    {"box_position": {"x": 89, "y": 501, "width": 124, "height": 16}, "text": "Fähigkeiten intakt?", "type": "NODE"},
    {"box_position": {"x": 587, "y": 501, "width": 124, "height": 18}, "text": "Fähigkeiten intakt?", "type": "NODE"},
    {"box_position": {"x": 373, "y": 513, "width": 54, "height": 12}, "text": "Falls NEIN", "type": "NO"},
    {"box_position": {"x": 273, "y": 523, "width": 26, "height": 14}, "text": "Falls", "type": "NODE"},
    {"box_position": {"x": 530, "y": 528, "width": 32, "height": 8}, "text": "Fallsla", "type": "NODE"},
    {"box_position": {"x": 65, "y": 531, "width": 38, "height": 14}, "text": "Falls JA", "type": "YES"},
    {"box_position": {"x": 273, "y": 537, "width": 28, "height": 12}, "text": "NEN", "type": "NODE"},
    {"box_position": {"x": 459, "y": 547, "width": 120, "height": 16}, "text": "Hat der Patient ein", "type": "NODE"},
    {"box_position": {"x": 641, "y": 547, "width": 128, "height": 16}, "text": "Hat der Patient eine", "type": "NODE"},
    {"box_position": {"x": 45, "y": 561, "width": 110, "height": 16}, "text": "Kann der Patient", "type": "NODE"},
    {"box_position": {"x": 275, "y": 565, "width": 54, "height": 14}, "text": "Sind dic", "type": "NODE"},
    {"box_position": {"x": 469, "y": 564, "width": 100, "height": 14}, "text": "Tracheostoma?", "type": "NODE"},
    {"box_position": {"x": 663, "y": 565, "width": 84, "height": 16}, "text": "orale Sonde?", "type": "NODE"},
    {"box_position": {"x": 63, "y": 579, "width": 72, "height": 14}, "text": "schreiben?", "type": "NODE"},
    {"box_position": {"x": 244, "y": 581, "width": 115, "height": 18}, "text": "grobmotorischen", "type": "NODE"},
    {"box_position": {"x": 461, "y": 591, "width": 36, "height": 14}, "text": "FalsIA", "type": "NODE"},
    {"box_position": {"x": 714, "y": 598, "width": 32, "height": 8}, "text": "Fallsla", "type": "NODE"},
    {"box_position": {"x": 241, "y": 599, "width": 124, "height": 16}, "text": "Fähigkeiten intakt?", "type": "NODE"},
    {"box_position": {"x": 21, "y": 613, "width": 36, "height": 14}, "text": "FalsJA", "type": "NODE"},
    {"box_position": {"x": 125, "y": 611, "width": 24, "height": 14}, "text": "Falls", "type": "NODE"},
    {"box_position": {"x": 447, "y": 613, "width": 124, "height": 16}, "text": "Sind die laryngealen", "type": "NODE"},
    {"box_position": {"x": 123, "y": 625, "width": 30, "height": 12}, "text": "NEIN", "type": "NODE"},
    {"box_position": {"x": 629, "y": 623, "width": 54, "height": 14}, "text": "Falls NEIN", "type": "NO"},
    {"box_position": {"x": 446, "y": 629, "width": 93, "height": 18}, "text": "Strukturen des", "type": "NODE"},
    {"box_position": {"x": 57, "y": 653, "width": 38, "height": 14}, "text": "Stelle", "type": "NODE"},
    {"box_position": {"x": 205, "y": 645, "width": 140, "height": 16}, "text": "Kommunikationsboard", "type": "NODE"},
    {"box_position": {"x": 447, "y": 646, "width": 106, "height": 18}, "text": "Patienten intakt?", "type": "NODE"},
    {"box_position": {"x": 41, "y": 669, "width": 70, "height": 16}, "text": "Papier und", "type": "NODE"},
    {"box_position": {"x": 203, "y": 663, "width": 38, "height": 18}, "text": "VOCA", "type": "NODE"},
    {"box_position": {"x": 461, "y": 671, "width": 36, "height": 14}, "text": "@1", "type": "NODE"},
    {"box_position": {"x": 61, "y": 687, "width": 30, "height": 14}, "text": "Stitt", "type": "NODE"},
    {"box_position": {"x": 205, "y": 685, "width": 148, "height": 20}, "text": "Computergestützte AAC", "type": "NODE"},
    {"box_position": {"x": 449, "y": 689, "width": 132, "height": 16}, "text": "Kann der Patient eine", "type": "NODE"},
    {"box_position": {"x": 29, "y": 705, "width": 26, "height": 14}, "text": "Zur", "type": "NODE"},
    {"box_position": {"x": 54, "y": 702, "width": 69, "height": 20}, "text": "Verfugung", "type": "NODE"},
    {"box_position": {"x": 449, "y": 707, "width": 110, "height": 14}, "text": "Deflation des Cuff", "type": "NODE"},
    {"box_position": {"x": 449, "y": 725, "width": 68, "height": 14}, "text": "tolerieren?", "type": "NODE"},
    {"box_position": {"x": 391, "y": 743, "width": 24, "height": 12}, "text": "Falls", "type": "NODE"},
    {"box_position": {"x": 604, "y": 748, "width": 48, "height": 8}, "text": "FallsNFIN", "type": "NODE"},
    {"box_position": {"x": 217, "y": 751, "width": 38, "height": 14}, "text": "Falls JA", "type": "YES"},
    {"box_position": {"x": 389, "y": 755, "width": 30, "height": 12}, "text": "NEN", "type": "NODE"},
    {"box_position": {"x": 487, "y": 749, "width": 38, "height": 12}, "text": "EIst4", "type": "NODE"},
    {"box_position": {"x": 478, "y": 768, "width": 51, "height": 17}, "text": "Einweg-", "type": "NODE"},
    {"box_position": {"x": 589, "y": 767, "width": 96, "height": 18}, "text": "Hat der Patient", "type": "NODE"},
    {"box_position": {"x": 109, "y": 791, "width": 124, "height": 18}, "text": "Computergestützte", "type": "NODE"},
    {"box_position": {"x": 287, "y": 793, "width": 74, "height": 16}, "text": "Blick- oder", "type": "NODE"},
    {"box_position": {"x": 479, "y": 787, "width": 78, "height": 16}, "text": "Sprechventil", "type": "NODE"},
    {"box_position": {"x": 589, "y": 787, "width": 106, "height": 18}, "text": "hartnäckige oder", "type": "NODE"},
    {"box_position": {"x": 109, "y": 809, "width": 30, "height": 14}, "text": "AAC", "type": "NODE"},
    {"box_position": {"x": 589, "y": 805, "width": 116, "height": 14}, "text": "reichliche Sekrete?", "type": "NODE"},
    {"box_position": {"x": 287, "y": 811, "width": 154, "height": 18}, "text": "Augenzwinkergesteuert", "type": "NODE"},
    {"box_position": {"x": 169, "y": 827, "width": 26, "height": 14}, "text": "mit", "type": "NODE"},
    {"box_position": {"x": 297, "y": 824, "width": 125, "height": 24}, "text": "computergestutzte", "type": "NODE"},
    {"box_position": {"x": 707, "y": 833, "width": 38, "height": 14}, "text": "Falls JA", "type": "YES"},
    {"box_position": {"x": 109, "y": 843, "width": 148, "height": 16}, "text": "Einzelschaltersteuerun", "type": "NODE"},
    {"box_position": {"x": 287, "y": 843, "width": 30, "height": 16}, "text": "AAC", "type": "NODE"},
    {"box_position": {"x": 489, "y": 841, "width": 118, "height": 16}, "text": "Sprecheinsatz der", "type": "NODE"},
    {"box_position": {"x": 631, "y": 843, "width": 24, "height": 12}, "text": "Falls", "type": "NODE"},
    {"box_position": {"x": 629, "y": 855, "width": 30, "height": 12}, "text": "NEIN", "type": "NODE"},
    {"box_position": {"x": 123, "y": 861, "width": 32, "height": 14}, "text": "oder", "type": "NODE"},
    {"box_position": {"x": 154, "y": 859, "width": 99, "height": 20}, "text": "Blicksteuerung", "type": "NODE"},
    {"box_position": {"x": 497, "y": 859, "width": 102, "height": 14}, "text": "Trachealkanüle", "type": "NODE"},
    {"box_position": {"x": 680, "y": 857, "width": 91, "height": 20}, "text": "Elektrolarynx", "type": "NODE"},
    {"box_position": {"x": 53, "y": 901, "width": 698, "height": 20}, "text": "Dokumentiere den Kommunikationsbedarf und den Erfolg von Kommunikationsinterventionen in der", "type": "NODE"},
    {"box_position": {"x": 351, "y": 921, "width": 98, "height": 16}, "text": "Patientenakte", "type": "NODE"},
    {"box_position": {"x": 86, "y": 936, "width": 630, "height": 24}, "text": "(bei mangelnder Effektivität und ausbleibendem Erfolg sollte eine alternative augmentative", "type": "NODE"},
    {"box_position": {"x": 241, "y": 959, "width": 318, "height": 18}, "text": "Kommunikationsmethode angeboten werden)", "type": "NODE"},
    {"box_position": {"x": 422, "y": 189, "width": 40, "height": 18}, "text": "quat?", "type": "NODE"}
]


    transformation_prompt = f"""
    Transformiere diese JSON-Daten eines medizinischen Flussdiagramms in eine logische Markdown-Hierarchie in Deutsch.
    Korrigiere OCR-Fehler (z.B. 'Fallsla' -> 'Falls Ja').

    JSON DATA:
    {str(json_data)}
    """

    try:
        response = model.generate_content(transformation_prompt)
        knowledge_base = response.text
        print("\n--- WISSENSBASIS ERSTELLT ---")
        print(knowledge_base)
    except Exception as e:
        print(f"Fehler bei der Generierung: {e}")

def medical_chatbot(user_query):
    if 'knowledge_base' not in globals():
        return "System nicht bereit."

    prompt = f"""
    DU BIST EIN ROBOTER, DER NUR AUS DER GEGEBENEN WISSENSBASIS ANTWORTET.

    WISSENSBASIS:
    {knowledge_base}

    ANWEISUNG:
    1. Suche den Startpunkt im Flussdiagramm ("Patient beatmet?").
    2. Folge den Pfaden (Falls JA / Falls NEIN) basierend auf der Nutzerfrage.
    3. Wenn der Nutzer sagt, die Reaktion ist NICHT adäquat, folge dem Pfad zu "Zunächst primäre und supportive Therapie".
    4. Wenn die Antwort NICHT in der Wissensbasis steht, sag: "Dazu enthält das Flussdiagramm keine Informationen."
    5. Erfinde KEINE medizinischen Notfallmaßnahmen wie 'Pneumothorax', wenn sie nicht im Text stehen.

    NUTZERFRAGE: {user_query}
    """
    res = model.generate_content(prompt)
    return res.text

if 'knowledge_base' in globals():
    print("\nTest-Antwort:", medical_chatbot("Was tun bei beatmeten Patienten?"))

In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output

text_input = widgets.Text(
    value='',
    placeholder='Frage hier eingeben (z.B. Was tun bei RASS -4?)...',
    description='Frage:',
    layout=widgets.Layout(width='70%')
)

button = widgets.Button(
    description='Absenden',
    button_style='success',
    tooltip='Klicken zum Senden',
    icon='paper-plane'
)

output = widgets.Output()

def on_button_clicked(b):
    with output:
        clear_output()
        user_query = text_input.value
        if not user_query:
            print("Bitte gib eine Frage ein.")
            return

        print(f"User: {user_query}")

        try:
            response = medical_chatbot(user_query)
            print(f"\nBot: {response}")
        except Exception as e:
            print(f"Fehler: {e}")

button.on_click(on_button_clicked)

print("--- Sample Flowchart BOT ---")
display(widgets.HBox([text_input, button]), output)